In [1]:
import numpy as np
np.set_printoptions(precision=3)

Dividing into blocks

In [2]:
def matrix_divide(A):
    if A.shape[0] != A.shape[1]:
        print "Matrix is not squared"
        return
    else:
        N = A.shape[0]
    
    A11 = A[:N/2,:N/2]
    A12 = A[:N/2,N/2:]
    A21 = A[N/2:,:N/2]
    A22 = A[N/2:,N/2:]
    
    Block_matrixes = [A11,A12,A21,A22]
    
    return Block_matrixes

A_blocks = matrix_divide(np.random.randn(4,4))
B_blocks = matrix_divide(np.random.randn(4,4))

Strassen recursive algorithm

In [3]:
def my_Strassen(A,B):
    
    N = len(A)
    
    min_dimension = 1024
    if N < min_dimension:
        C = np.dot(A,B)
    else:
        Aij = matrix_divide(A)
        Bij = matrix_divide(B)
    
        f1 = my_Strassen((Aij[0] + Aij[3]),(Bij[0] + Bij[0]))
        f2 = my_Strassen((Aij[2] + Aij[3]),Bij[0])
        f3 = my_Strassen(Aij[0],(Bij[1] - Bij[3]))
        f4 = my_Strassen(Aij[3],(Bij[2] - Bij[0]))
        f5 = my_Strassen((Aij[0] + Aij[1]),Bij[3])
        f6 = my_Strassen((Aij[2] - Aij[0]),(Bij[0] + Bij[1]))
        f7 = my_Strassen((Aij[1] - Aij[3]),(Bij[2] + Bij[3]))

        c11 = f1 + f4 - f5 + f7
        c12 = f3 + f5
        c21 = f2 + f4
        c22 = f1 - f2 + f3 + f6
    
        C_up = np.append(c11,c12,axis = 1)
        C_down = np.append(c21,c22,axis = 1)
        C = np.append(C_up,C_down,axis = 0)
    return C

N = 2*2048
A = 2*np.random.rand(N,N)
B = 3*np.random.rand(N,N)
C1 = np.dot(A,B)
C2 = my_Strassen(A,B)

print 'Relative error =', (np.linalg.norm(C1 - C2)) / np.linalg.norm(C1)

Relative error = 0.0486290998232


In [4]:
def pure_loops_MM(A,B):
    N = A.shape[0]   
    args = np.arange(N)
    C = np.zeros((N,N),dtype = np.double)
    for i in args:
        for j in args:
            for k in args:
                C[i][j] = C[i][j] + A[i][k]*B[k][j]
    return C
     

In [5]:
def loops_MM(A,B):
    min_dimension = 1024
    N = A.shape[0]
    if N < min_dimension:
        args = np.arange(N)
        C = pure_loops_MM(A,B)
    else:
        Aij = matrix_divide(A)
        Bij = matrix_divide(B)  
        
        c11 = loops_MM(Aij[0],Bij[0]) + loops_MM(Aij[1],Bij[2])
        c12 = loops_MM(Aij[0],Bij[1]) + loops_MM(Aij[1],Bij[3])
        c21 = loops_MM(Aij[2],Bij[0]) + loops_MM(Aij[3],Bij[2])
        c22 = loops_MM(Aij[3],Bij[3]) + loops_MM(Aij[2],Bij[1])
        
        C_up = np.append(c11,c12,axis = 1)
        C_down = np.append(c21,c22,axis = 1)
        C = np.append(C_up,C_down,axis = 0)
        
        
        
    return C

N = 128
A = 2*np.random.randn(N,N)
B = 3*np.random.randn(N,N)
%timeit C1 = np.dot(A,B)
%timeit C2 = loops_MM(A,B)
%timeit C3 = pure_loops_MM(A,B)

print 'Relative error =', (np.linalg.norm(C1 - C2)) / np.linalg.norm(C1)

1000 loops, best of 3: 226 µs per loop
1 loops, best of 3: 13.8 s per loop
1 loops, best of 3: 6.5 s per loop
Relative error = 0.0486290998232


Timings

In [6]:
from timeit import default_timer as timer

In [9]:
dims = [256,512, 1024, 2048]
bench_names = ['numpy', 'my Strassen',"loops"]
bench_funcs = [np.dot, my_Strassen, loops_MM]
timings = {bench_name: [] for bench_name in bench_names}

for N in dims:
    print "N = ",N
    A = 2*np.random.rand(N,N)
    B = 3*np.random.rand(N,N)
    for bench_name, bench_func in zip(bench_names, bench_funcs):
        print '\nMeasuring {}'.format(bench_func)
        start_time = timer()
        bench_func(A,B)
        time_delta = timer() - start_time
        timings[bench_name].append(time_delta)


N =  256

Measuring <built-in function dot>

Measuring <function my_Strassen at 0x00000000041687B8>

Measuring <function loops_MM at 0x0000000004168828>
N =  512

Measuring <built-in function dot>

Measuring <function my_Strassen at 0x00000000041687B8>

Measuring <function loops_MM at 0x0000000004168828>
N =  1024

Measuring <built-in function dot>

Measuring <function my_Strassen at 0x00000000041687B8>

Measuring <function loops_MM at 0x0000000004168828>
N =  2048

Measuring <built-in function dot>

Measuring <function my_Strassen at 0x00000000041687B8>

Measuring <function loops_MM at 0x0000000004168828>


KeyboardInterrupt: 

Plotting

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline 

In [10]:
plt.figure(figsize=(10,8))

figs = [ plt.semilogy(dims, values, label=bench_name)\
        for bench_name, values in timings.iteritems()];

ax0, = figs[0]
ax0.set_marker('o')

ax1, = figs[1]
ax1.set_marker('s')

ax2, = figs[2]
ax2.set_marker('b')


    
plt.legend(loc='best')
plt.title('Benchmarking results with best of timeit', y=1.03)
plt.xlabel('Matrix dimension size')
plt.ylabel('Time, s')
#plt.axis([dims[0]-100,dims[len(dims)-1]+100,timings_best.iteritems()])

NameError: name 'plt' is not defined

Complexity evaluation of 2steps of Strassen algorithm

$A_{2strassens}(n) = 7A_{strassen}(\frac{n}{2}) + 18(\frac{n}{2})^2 = \\
= 7(7A_{naiv}(\frac{n}{4}) + 18(\frac{n}{4})^2) + 18(\frac{n}{2})^2 =  
\lbrace\begin{aligned}
1.5321 n^3 & \text{for naive}\\
\\0.765625 n^3 & \text{for naive with divide and conquere}\\
\end{aligned}
$